In [1]:
import numpy as np
import h5py
import time 
import copy
from scipy import signal
file_name = "../data/MNISTdata.hdf5"
data = h5py.File(file_name, "r")
x_train = np.float32(data["x_train"][:]).reshape(-1, 28, 28)
y_train = np.int32(np.hstack(np.array(data["y_train"]))).reshape(-1,1)
x_test = np.float32(data["x_test"][:]).reshape(-1, 28, 28)
y_test = np.int32(np.hstack(np.array(data["y_test"]))).reshape(-1,1)
data.close()

In [2]:
class CNN():
    def __init__(self, x_train, y_train, x_test, y_test, num_channels=1, learning_rate=0.01, num_epochs=5):
        self.x_train = x_train
        self.x_test = x_test
        self.y_train = y_train
        self.y_test = y_test
        
        self.num_outputs = 10
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.num_channels = num_channels
        self.d = self.x_train.shape[1]
        
        self.params = {}
        r = np.random.RandomState(1234)
        self.params["K"] = r.randn(self.num_channels, 3, 3) / self.d
        self.ky = self.params["K"].shape[1]
        self.kx = self.params["K"].shape[2]
        
        # channels = 1
        self.params["W"] = r.randn(self.num_outputs, 
                                   self.d - self.ky + 1,
                                   self.d - self.kx + 1) / self.d
        self.params["b"] = np.zeros((self.num_outputs, 1))
        
        self.gradients = {}
        
        print("training sample size: [{}]\ntest sample size:[{}]\nchannels:[{}]".format(self.x_train.shape, self.x_test.shape, self.num_channels))


    def convolution_process(self, img):
        convoluted = np.zeros((self.params["K"].shape[0],
                               self.d - self.ky + 1, 
                               self.d - self.kx + 1 ))
        for filter_idx in range(self.params["K"].shape[0]):
            convoluted[filter_idx, :, :] = signal.correlate2d(img[0,:,:], self.params["K"][filter_idx, :, :], mode='valid', boundary='wrap')
        return convoluted
        
    def relu(self, Z):
        U = copy.deepcopy(Z)
        U[U<=0] = 0
        return  U

    def relu_gradient(self, Z):
        dZ = copy.deepcopy(Z)
        dZ[dZ >= 0] = 1
        dZ[dZ < 0] = 0
        return  dZ

    def softmax(self, U):
        temp = np.exp(U)
        return temp / np.sum(temp)

    def forward_propagation(self):
        random_index = np.random.randint(self.x_train.shape[0])
        self.img = self.x_train[random_index].reshape((1, self.d, self.d))
        self.img_label = self.y_train[random_index].reshape((-1,1))
        self.forward_results = {}
        self.forward_results["Z"] = self.convolution_process(self.img)
        self.forward_results["H"] = self.relu(self.forward_results["Z"])
        self.forward_results["U"] = np.sum(
            np.multiply(self.params["W"],self.forward_results["H"]), 
            axis=(1,2)).reshape((self.num_outputs ,1)) + self.params["b"]
        self.forward_results["S"] = self.softmax(self.forward_results["U"])

    def back_propagation(self):
        ey = np.zeros((self.num_outputs, 1)); ey[self.img_label] = 1
        self.gradients["dU"] = - (ey - self.forward_results["S"])
        self.gradients["db"] = self.gradients["dU"]
        self.gradients["delta"] = np.tensordot(self.gradients["dU"].squeeze(), self.params["W"], axes=1)
        self.gradients["dW"] = np.tensordot(self.gradients["dU"].squeeze(), self.forward_results["H"][0,:,:], axes=0)
        dsigmaZ = self.relu_gradient(self.forward_results["Z"])
        temp = np.multiply(dsigmaZ, self.gradients["delta"])
        self.gradients["dK"] = signal.correlate2d(self.img[0, :,:], temp[0,:,:], mode='valid',  boundary='wrap')
                                                         
    def train(self):
        for epoch in range(self.num_epochs):
            if (epoch > 5):
                self.learning_rate = 0.001
            if (epoch > 10):
                self.learning_rate = 0.0001
            if (epoch > 15):
                self.learning_rate = 0.00001
            total_correct = 0
            for i in range(int(self.x_train.shape[0])):
                self.forward_propagation()
                prediction_train =  np.argmax(self.forward_results["S"], axis=0)
                total_correct += np.sum(prediction_train == self.img_label)
                self.back_propagation()
                self.params["b"] -= self.learning_rate * self.gradients["db"]
                self.params["W"] -= self.learning_rate * self.gradients["dW"]
                self.params["K"] -= self.learning_rate * self.gradients["dK"]
            print("epoch:{} | Training Accuracy:[{}]".format(epoch+1, total_correct/(self.x_train.shape[0])))
#     def test(self):
        
#         self.Z = self.convolution_process(self.img)
#         self.forward_results["H"] = self.relu(self.forward_results["Z"])
#         self.forward_results["U"] = np.zeros((self.num_channels,self.num_outputs, 1))
#         for i in range(self.num_outputs):
#             # to modify in the future
#             self.forward_results["U"][1,i,1] = np.tensordot(self.params["W"][i, :, :],
#                                                           self.forward_results["H"],
#                                                           axes=((0,1),(0,1))
#                                                          ) + self.params["b"][i,1]
#         # to modify in the future
#         self.forward_results["S"] = self.softmax(self.forward_results["U"][1,:,:])


#         self.prediction = np.apply_along_axis(np.argmax, 0, self.S)
#         correct_ratio = np.mean(self.prediction == self.y_test)
#         return correct_ratio

In [3]:
myCNN = CNN(x_train, y_train, x_test, y_test,  num_channels=1, learning_rate=0.01, num_epochs=5)

training sample size: [(60000, 28, 28)]
test sample size:[(10000, 28, 28)]
channels:[1]


In [4]:
myCNN.train()
#myCNN.forward_propagation()
# myCNN.back_propagation()

epoch:1 | Training Accuracy:[0.8928166666666667]
epoch:2 | Training Accuracy:[0.9150666666666667]
epoch:3 | Training Accuracy:[0.9183666666666667]
epoch:4 | Training Accuracy:[0.9211666666666667]
epoch:5 | Training Accuracy:[0.9224833333333333]


In [13]:
def convolution(img, myfilter):
    ky, kx = myfilter.shape
    d = img.shape[1]
    Z = np.zeros((d - ky + 1, d - kx + 1))
    for i in range(Z.shape[0]):
        for j in range(Z.shape[1]):
            Z[i,j] = np.tensordot(img[0, i:(i+ky), j:(j+kx)], myfilter, axes=((0,1),(0,1)))
    return Z

In [21]:
img = np.array([[1,2,3], [4,5,6], [7,8,9]]).reshape((1,3,3))
myfilter = np.array([[1,1], [2,2]])
convolution(img, myfilter)

array([[21., 27.],
       [39., 45.]])

In [19]:
img = np.array([[1,2,3], [4,5,6]]).reshape((1,2,3))
img.shape

(1, 2, 3)

In [35]:
signal.correlate2d(img[0,:,:], myfilter, mode='valid',  boundary='wrap')

array([[21, 27],
       [39, 45]])

In [25]:
img[0,:,:]

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [2]:
a = np.float32(np.array([[-1,0,3], [4,5,-6], [7,8,-9], [2, -5,9]]).reshape((2,3,2)))
a

array([[[-1.,  0.],
        [ 3.,  4.],
        [ 5., -6.]],

       [[ 7.,  8.],
        [-9.,  2.],
        [-5.,  9.]]], dtype=float32)

In [15]:
a[a == 0] = 1e-8
a[a >=0] = 1
a[a< 0] = 0

In [16]:
a

array([[[0., 1.],
        [1., 1.],
        [1., 0.]],

       [[1., 1.],
        [0., 1.],
        [0., 1.]]], dtype=float32)

In [5]:
def relu(Z):
    """
    Z: must be of size (num_channels, d - ky + 1, d - kx + 1)
    """
    U = copy.deepcopy(Z)
    U[U<=0] = 0
    return  U
relu(a)

array([[[0., 0.],
        [3., 4.],
        [5., 0.]],

       [[7., 8.],
        [0., 2.],
        [0., 9.]]], dtype=float32)

In [22]:
x_train.shape[0]

60000